In [139]:
import numpy as np
import math
import networkx as nx
import random

In [140]:
class Tank():
    def __init__(self, tank_id, current_load, max_load, consumption_rate):
        self.id = tank_id
        self.load = current_load
        self.max_load = max_load
        self.rate = consumption_rate
   
    def fill(self):
        self.load = self.max_load    
        
    def partial_fill(self, fill_percentage):
        self.load = self.load + self.max_load * fill_percentage
    
    def tank_extra_capacity(self):
        return(self.max_load - self.load)
        
       
        

In [141]:
class Truck():
    def __init__(self, truck_id, current_load, max_load, current_position, load_fractions_deliverable):
        self.id = truck_id
        self.load = current_load
        self.max_load = max_load
        self.pos = current_position
        self.fractions = load_fractions_deliverable
        
    def fill(self):
        self.load = self.max_load
        
    def deliver(self, fraction_id: int):
        self.load = self.load - self.fractions[fraction_id] * self.max_load
    
    def possible_delivery_quantities(self, tank_extra_capacity):
        all_delivery_quantities = self.load * self.fractions
        return(all_delivery_quantities[ all_delivery_quantities <= tank_extra_capacity])
       
    

In [142]:
class System():
    def __init__(self, tanks, trucks, adjacency_matrix, weights_matrix):
        self.tanks = tanks
        self.trucks = trucks
        self.graph = adjacency_matrix
        self.weights = weights_matrix
        self.k = len(trucks)
        self.n = len(tanks)
        self.s = self.state()
        
    def truck_loads(self):
        return([self.trucks[i].load for i in range(self.k)])
    
    def truck_positions(self):
        return([self.trucks[i].pos for i in range(self.k)])
    
    def tank_loads(self):
        return([self.tanks[i].load for i in range(self.n)])
    
    def state(self):
        #[ positions, truck-loads, tank-loads]
        s = [self.truck_positions(), self.truck_loads(), self.tank_loads()]
        return(s)
    
    def update_state(self):
        self.s = self.state()
        
    def random_action(self, seed = None):
        if seed != None:
            random.seed(42)
        # Choose a position for each truck randomly
        
        #CORRE GIR EL CODIGO, PLANTEAMIENTO PARA QUE FUNCIONE
        possible_positions_index = np.isin(self.graph, 1)
        possible_positions = np.where(possible_positions_index)
        print("possible_positions:", possible_positions)
        for i in range(self.k):
            random.randint(len(possible_positions[0]))
            new_position = random.choice(zip(possible_positions), 1)
            print("new position: ",new_position)
            self.trucks[i].pos = new_position
            
        # Choose a new (possible) load delivery for each truck to the new tank (position)
        # and update the tank's load after deliverying the chosen quantity.
        for i in range(self.k):
            truck_pos = self.trucks[i].pos
            print(truck_pos)
            if truck_pos == self.n:
                continue;
            else:
                current_truck = trucks[i]
                current_tank = tanks[truck_pos]
                current_extra_tank_capacity = current_tank.tank_extra_capacity()
                possible_delivery_quantities = current_truck(current_extra_tank_capacity)
                delivery_quantity = np.random.choice(possible_delivery_quantities)
                current_tank.load = current_tank.load + delivery_quantity
            
        self.update_state()
        
            

            


In [143]:
random.randint(0,100)
print(np.__version__)



1.13.1


In [144]:
# A simple system's adjacency matrix
n = 5

def simple_graph(n: int):    
    A = np.zeros((n,n))
    A[0,0:n] = 1
    A = A.astype(int)
    return(A)

A = simple_graph(n+1)

print(A)

G = nx.from_numpy_matrix(np.array(A)) 
nx.draw(G, with_labels=True)



[[1 1 1 1 1 1]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]


C:\Users\Daniel\Anaconda3\lib\site-packages\networkx\drawing\nx_pylab.py:126: MatplotlibDeprecationWarning: pyplot.hold is deprecated.
    Future behavior will be consistent with the long-time default:
    plot commands add elements without first clearing the
    Axes and/or Figure.
  b = plt.ishold()
C:\Users\Daniel\Anaconda3\lib\site-packages\networkx\drawing\nx_pylab.py:138: MatplotlibDeprecationWarning: pyplot.hold is deprecated.
    Future behavior will be consistent with the long-time default:
    plot commands add elements without first clearing the
    Axes and/or Figure.
  plt.hold(b)
C:\Users\Daniel\Anaconda3\lib\site-packages\matplotlib\__init__.py:917: UserWarning: axes.hold is deprecated. Please remove it from your matplotlibrc and/or style files.
  warnings.warn(self.msg_depr_set % key)
C:\Users\Daniel\Anaconda3\lib\site-packages\matplotlib\rcsetup.py:152: UserWarning: axes.hold is deprecated, will be removed in 3.0
  warnings.warn("axes.hold is deprecated, will be remove

In [145]:
possible_positions_index = np.isin(A, 1)
possible_positions = np.where(possible_positions_index)
possible_positions

(array([0, 0, 0, 0, 0, 0], dtype=int64),
 array([0, 1, 2, 3, 4, 5], dtype=int64))

In [146]:
# Tanks' information
n = 5 
tank_ids = list(range(n))
tank_max_loads =  np.array([100, 100., 200., 300., 400.])
tank_current_loads =  np.array([50, 60., 120., 150., 300.])
tank_consumption_rates =  np.array([2] * n)


In [147]:
# Trucks' information
k = 2
truck_ids = list(range(k))
truck_max_loads = np.array([20., 50.])
truck_current_loads = truck_max_loads.copy()
truck_current_positions =  np.array([n] * k)
truck_fractions_deliverable =  np.array([1.] * k) # we for now we only allow to deliver all the content of the truck


In [148]:
# System's information
graph = simple_graph(n+1)
tanks = [Tank( tank_id, current_load, max_load, consumption_rate ) 
         for  tank_id, current_load, max_load, consumption_rate in 
         zip( tank_ids, tank_current_loads, tank_max_loads, tank_consumption_rates)]
trucks = [Truck( truck_id, current_load, max_load, current_position, load_fractions_deliverable ) 
         for  truck_id, current_load, max_load, current_position, load_fractions_deliverable in 
         zip(truck_ids, truck_current_loads, truck_max_loads, truck_current_positions, 
             truck_fractions_deliverable)]

def simple_weights(n:int, w: float):    
    W = np.full((n,n), np.inf)
    W[0,:] = w
    return(W)
w =  np.array([0, -20., -10., -30., -50.5, -45.])

weights_matrix = simple_weights(n+1, w)

toy_system = System(tanks, trucks, graph, weights_matrix)


In [149]:
for i in range(n):
    print(tanks[i].id) 

#print(toy_system.truck_loads())
print(toy_system.tank_loads())
print(toy_system.truck_positions())

print(toy_system.state())
print(toy_system.weights)
#POSITION 'n' is the position of the Charge location (node 0 in the plot) ???? clarify notation and code

0
1
2
3
4
[50.0, 60.0, 120.0, 150.0, 300.0]
[5, 5]
[[5, 5], [20.0, 50.0], [50.0, 60.0, 120.0, 150.0, 300.0]]
[[  0.  -20.  -10.  -30.  -50.5 -45. ]
 [  inf   inf   inf   inf   inf   inf]
 [  inf   inf   inf   inf   inf   inf]
 [  inf   inf   inf   inf   inf   inf]
 [  inf   inf   inf   inf   inf   inf]
 [  inf   inf   inf   inf   inf   inf]]


In [150]:
toy_system.random_action()
print(toy_system.state())

possible_positions: (array([0, 0, 0, 0, 0, 0], dtype=int64), array([0, 1, 2, 3, 4, 5], dtype=int64))


TypeError: choice() takes 2 positional arguments but 3 were given

In [ ]:
print(toy_system.graph)

In [ ]:
[trucks[j].load for j in range(k)]

In [ ]:
k

In [ ]:
truck_current_positions 

In [ ]:

trucks[0].possible_delivery_quantities()